In [1]:
import requests
import yaml
import pandas as pd
with open('../config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Access the API key
api_key = config['elsevier_api']['api_key']

In [2]:
def get_all_doi_journal(journal_issn, end_volume=200):
    title_doi_list = []
    # Specify the volume you want to query
    for volume_number in range(0,end_volume):
        # API Endpoi
        # nt for searching articles in Scopus based on volume and ISSN
        url = "https://api.elsevier.com/content/search/scopus"

        # Headers: Include API key and specify JSON response
        headers = {
            "X-ELS-APIKey": api_key,
            "Accept": "application/json",
        }

        # Pagination control
        start = 0  # Start index of results
        count = 20  # Number of results per page (max is 25)
        # Pagination control
        start = 0  # Start index of results
        count = 20  # Number of results per page (max is 25)

        # Loop to paginate through results
        while True:
            # Query parameters: Filter articles by journal ISSN and volume number
            params = {
                "query": f"ISSN({journal_issn}) AND VOLUME({volume_number})",
                "view": "COMPLETE",  # Request complete metadata for the articles
                "count": count,  # Number of results per page
                "start": start,  # Start index of results for pagination
            }

            try:
                # Send the request to the API
                response = requests.get(url, headers=headers, params=params)

                # Check if the response is successful
                if response.status_code == 200:
                    data = response.json()

                    # Extract article entries from the response
                    entries = data.get("search-results", {}).get("entry", [])

                    # If no more entries, break the loop
                    if not entries:
                        print(f"Volume {volume_number} done.")
                        break

                    if len(entries) == 1:
                        doi = entries[0].get("prism:doi", "No DOI")
                        if doi == "No DOI":
                            break

                    # print(f"Found {len(entries)} articles starting at index {start}.\n")

                    # Iterate over articles and print their titles and DOIs
                    for entry in entries:
                        title = entry.get("dc:title", "No Title")
                        doi = entry.get("prism:doi", "No DOI")
                        date = entry.get("prism:coverDate", "No Date")
                        # make sure it's not "No DOI"
                        if doi != "No DOI":
                            title_doi_list.append({"title": title, "doi": doi, "volume": volume_number, "date": date})
                        # print(f"Title: {title}\nDOI: {doi}\n")

                    # Increment the start index for the next batch of articles
                    start += count

                elif response.status_code == 403:
                    print("Access Denied. Check if you have the proper entitlements.")
                    break  # Stop if access is denied

                else:
                    print(f"Failed to retrieve articles. Status Code: {response.status_code}")
                    print("Response:", response.text)
                    break  # Stop on other failures

            except Exception as e:
                print(f"An error occurred: {e}")
                break  # Stop on exceptions
        if  volume_number %10 == 0:
            print(f"{len(title_doi_list)} records done")
            title_doi_dataframe = pd.DataFrame(title_doi_list)
            title_doi_dataframe.to_csv(f"../journal-meta/{journal_issn}_doi.csv", index=False)
    title_doi_dataframe = pd.DataFrame(title_doi_list)
    title_doi_dataframe.to_csv(f"../journal-meta/{journal_issn}_doi.csv", index=False)
    return title_doi_dataframe

In [ ]:
# TRC
title_doi_dataframe = get_all_doi_journal("0968-090X", 200)

In [ ]:
# TRA
title_doi_dataframe = get_all_doi_journal("0965-8564", 200)

In [ ]:
# TRB
title_doi_dataframe = get_all_doi_journal("0191-2615", 200)

In [ ]:
# TRD
title_doi_dataframe = get_all_doi_journal("1361-9209", 200)

In [ ]:
# TRE
title_doi_dataframe = get_all_doi_journal("1366-5545", 200)

In [ ]:
# TRF
title_doi_dataframe = get_all_doi_journal("1369-8478", 200)

In [ ]:
# TRIP
title_doi_dataframe = get_all_doi_journal("2590-1982", 30)